In [61]:
%pip install qiskit
%pip install qiskit-aer

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [56]:
from math import pi
# Import the Qiskit SDK
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister
from qiskit import execute, Aer


In [57]:
# Quantum Fourier transform of |q>, of length n.
# See Fig. 3 of Khosropour et al. (2011) for a circuit diagram.
def qft(circ, q, n):
    # Iterate through the target.
    for i in range(n,0,-1):
        # Apply the Hadamard gate to the target.
        circ.h(q[i-1])

        # Iterate through the control.
        for j in range(i-1,0,-1):
            circ.cp(2*pi/2**(i-j+1), q[j-1], q[i-1])

# Inverse Fourier transform of |q>, of length n.
def iqft(circ, q, n):
    # Iterate through the target.
    for i in range(1,n+1):
        # Iterate through the control.
        for j in range(1,i):
            # The inverse Fourier transform just uses a negative phase.
            circ.cp(-2*pi/2**(i-j+1), q[j-1], q[i-1])

        # Apply the Hadamard gate to the target.
        circ.h(q[i-1])
        
def add(circ, a, b, n):
    # move n forward by one to account for overflow
    n += 1

    # Take the QFT.
    qft(circ, b, n)

    # Compute controlled-phases.
    # Iterate through the targets.
    for i in range(n,0,-1):
        # Iterate through the controls.
        for j in range(i,0,-1):
            # If the qubit a[j-1] exists run cu1, if not assume the qubit is 0 and never existed
            if len(a) - 1 >= j - 1:
                circ.cp(2*pi/2**(i-j+1), a[j-1], b[i-1])

    # Take the inverse QFT.
    iqft(circ, b, n)

In [65]:
def qadd_num(first, second, N):
    
    a = QuantumRegister(N)
    b = QuantumRegister(N+1)

    ca = ClassicalRegister(N)
    cb = ClassicalRegister(N+1)

    qc = QuantumCircuit(a, b, ca, cb)

    for i in range(0, N+1):
        if first[N-i] == "1":
            qc.x(a[i])

    for i in range(0, N+1):
        if second[N-i] == "1":
            qc.x(b[i])
    
    add(qc, a, b, N)

    qc.measure(a, ca)
    qc.measure(b, cb)

    print(qc.draw(scale=0.5))
    
    backend_sim = Aer.get_backend('qasm_simulator')
    job_sim = execute(qc, backend_sim)
    result_sim = job_sim.result()

    x = result_sim.get_counts(qc)

    return list(x.keys())[0].split(" ")[0]

In [66]:
def find_the_primes_numbers(target, arr):
    n = len(arr)
    for i in range(n):
        for j in range(i+1, n):
            first = f'{arr[i]:07b}'
            second = f'{arr[j]:07b}'
            qsum = qadd_num(first, second, 6)

            if int(qsum, 2) == target:
                return arr[i], arr[j]
    return -1

In [69]:
A = find_the_primes_numbers (24,[1,3,5,7,11,13,15])
print(f"\n Answer is : {A}")

        ┌───┐                                                                 »
q828_0: ┤ X ├─────────────────────────────────────────────────────────────────»
        └───┘                                                                 »
q828_1: ──────────────────────────────────────────────────────────────────────»
                                                                              »
q828_2: ──────────────────────────────────────────────────────────────────────»
                                                                              »
q828_3: ──────────────────────────────────────────────────────────────────────»
                                                                              »
q828_4: ──────────────────────────────────────────────────────────────────────»
                                                                              »
q828_5: ──────────────────────────────────────────────────────────────────────»
        ┌───┐                           